In [3]:
import os
import docx
import openai
import pinecone
from openai.embeddings_utils import get_embedding
from getpass import getpass

openai.api_key = getpass()

 ········


In [10]:
docs_path = "book"

## Parse document

In [12]:
text_chunks = []
for f_name in os.listdir(docs_path):
    doc_path = os.path.join(docs_path, f_name)
    doc = docx.Document(doc_path)
    for p in doc.paragraphs:
        text_chunks.append(p.text)

In [13]:
# remove all chunks shorter than 10 words and strip the rest
text_chunks = [string.strip().strip('\n') for string in text_chunks if len(string.split()) >= 10]

## Generate embeddings

In [14]:
chunks_with_embeddings = []
for chunk in text_chunks:
    embedding = get_embedding(chunk, engine="text-embedding-ada-002")
    chunks_with_embeddings.append({"text": chunk, "embedding": embedding})

## Pinecone

In [76]:
pinecone.init(
    api_key = getpass(),
    environment="us-central1-gcp"
)

 ········


In [16]:
# create index
index_name = "livro-python"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536)
    
# connect to index
index = pinecone.Index(index_name)

In [34]:
# process everything in batches of 64
batch_size = 64

for i in range(0, len(chunks_with_embeddings), batch_size):
    data_batch = chunks_with_embeddings[i: i+batch_size]
    
    # set end position of batch
    i_end = min(i+batch_size, len(chunks_with_embeddings))
    
    # get batch meta
    text_batch = [item["text"] for item in data_batch]
    
    # get ids
    ids_batch = [str(n) for n in range(i, i_end)]
    
    # get embeddings
    embeds = [item["embedding"] for item in data_batch]
    
    # prepare metadata and upsert batch
    meta = [{"text": text_batch} for text_batch in zip(text_batch)]
    to_upsert = zip(ids_batch, embeds, meta)
    
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))

## Query Index

In [38]:
def search_docs(query):
    xq = openai.Embedding.create(input=query, engine="text-embedding-ada-002")["data"][0]["embedding"]
    res = index.query([xq], top_k=5, include_metadata=True)
    chosen_text = []
    for match in res["matches"]:
        chosen_text = match["metadata"]
    return res["matches"]

In [73]:
matches = search_docs("A função raw_input do Python2 funciona no Python3 ?")
for match in matches:
    print(f"{match['score']:.2f}: {match['metadata']}")

0.92: {'text': ['A função raw_input do Python2 foi renomeada para input() no Python3:']}
0.86: {'text': ['O Python possui uma função que captura a entrada de valores: a função input() . Quando essa função é chamada, o programa para e espera o usuário digitar alguma coisa. Quando o usuário aperta a tecla ENTER , o programa processa e imprime o valor digitado em forma de string:']}
0.84: {'text': ['Existem casos que exigem o Python2 ao invés do Python3 como implementar algo em um ambiente que o programador não controla ou quando precisa utilizar algum pacote/módulo específico que não possui versão compatível com Python3. Vale ressaltar para quem deseja utilizar uma implementação alternativa do Python, como o IronPython ou Jython, que o suporte ao Python3 ainda é bastante limitado.']}
0.83: {'text': ['Para que várias ferramentas disponíveis na versão 3 funcionem na versão 2, ou seja, o módulo __future__   permite usar funcionalidades do Python3 no Python2. Mas cuidado, algumas funcionalid

## Prompt

In [77]:
def construct_prompt(query):
    matches = search_docs(query)
    
    chosen_text = []
    for match in matches:
        chosen_text.append(match["metadata"]["text"])
        
    prompt = """Responda à pergunta com a maior sinceridade possível usando o contexto abaixo e,
                se a resposta não for dentro do contexto, diga apenas: eu não sei."""
    prompt += "\n\n"
    prompt += "Contexto: " + "\n".join(str(text) for text in chosen_text)
    prompt += "\n\n"
    prompt += "Pergunta: " + query
    prompt += "\n"
    prompt += "Resposta: "
    return prompt

In [71]:
def answer_question(query):
    prompt = construct_prompt(query)
    res = openai.Completion.create(
        prompt=prompt,
        model="text-davinci-003",
        max_tokens=2000,
        temperature=0.0
    )
    
    return res.choices[0].text

In [74]:
print(answer_question("A função raw_input do Python2 funciona no Python3 ?"))

 Não, a função raw_input do Python2 foi renomeada para input() no Python3.
